In [1]:
%matplotlib inline

import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import tensorflow as tf

In [2]:
X_y = pickle.load(open("../datasets/notMNIST/all_image_array_train.pickle", "rb"))

In [3]:
X_y_test = pickle.load(open("../datasets/notMNIST/all_image_array_test.pickle", "rb"))

In [4]:
X_y.shape

(10010, 785)

In [5]:
X_y[1:10,0]

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.])

In [6]:
X_y[1:10,0][:,None]

array([[ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.]])

In [7]:
enc = OneHotEncoder()
enc.fit(X_y[:,0][:,None])
y = np.asarray(enc.transform(X_y[:,0][:,None]).toarray())
y_test = np.asarray(enc.transform(X_y_test[:,0][:,None]).toarray())

In [8]:
y.shape

(10010, 10)

In [9]:
y_test.shape

(10009, 10)

In [10]:
y

array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.]])

In [11]:
X = X_y[:,1:785]
X_y_test = X_y_test[:,1:785]

In [12]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.33, random_state=42)

In [13]:
graph = tf.Graph()
with graph.as_default():
    X_train_tf = tf.cast(tf.constant(X_train), tf.float32)
    y_train_tf = tf.cast(tf.constant(y_train), tf.float32)
    
    X_valid_tf = tf.cast(tf.constant(X_valid), tf.float32)
    
    X_test_tf = tf.cast(tf.constant(X_y_test), tf.float32)

    norm_dist_weights = tf.truncated_normal([X.shape[1], y.shape[1]])

    norm_dist_weights_tf = tf.Variable(norm_dist_weights)

    norm_dist_weights

    y.shape[1]

    biases_tf = tf.Variable(tf.zeros([y.shape[1]]))

    X_train.shape

    logits = tf.matmul(X_train_tf, norm_dist_weights_tf) + biases_tf
    tf_loss_alldata_perclass = tf.nn.softmax_cross_entropy_with_logits(logits, y_train_tf)
    print (tf_loss_alldata_perclass)
    loss = tf.reduce_mean(tf_loss_alldata_perclass)
    print(loss)
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    y_train_prediction = tf.nn.softmax(logits)
    
    y_valid_prediction = tf.nn.softmax(
    tf.matmul(X_valid_tf,norm_dist_weights_tf) + biases_tf
    )

    y_test_prediction = tf.nn.softmax(
    tf.matmul(X_test_tf,norm_dist_weights_tf) + biases_tf
    )
    
    

Tensor("SoftmaxCrossEntropyWithLogits:0", shape=(6706,), dtype=float32)
Tensor("Mean:0", shape=(), dtype=float32)


In [14]:
X_train_tf

<tf.Tensor 'Cast:0' shape=(6706, 784) dtype=float32>

In [15]:
norm_dist_weights

<tf.Tensor 'truncated_normal:0' shape=(784, 10) dtype=float32>

In [16]:
logits

<tf.Tensor 'add:0' shape=(6706, 10) dtype=float32>

In [17]:
y_train_tf

<tf.Tensor 'Cast_1:0' shape=(6706, 10) dtype=float32>

In [18]:
num_steps = 801

def accuracy(predictions, label):
    return(100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(label, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print ("Initialized")
    
    for step in range(num_steps):
        _, curr_loss, predictions = session.run([optimizer, loss, y_train_prediction])
        # below code is required for display purpose only
        
        if(step % 100 == 0):
            print('loss at step %d: %.4f' % (step, curr_loss))
            print('Training accuracy: %.2f%%' % accuracy(predictions, y_train))
            print('Validation accuracy: %.2f%%' % accuracy(y_valid_prediction.eval(), y_valid))
            print('Testing accuracy: %.2f%%' % accuracy(y_test_prediction.eval(), y_test))
            

Initialized
loss at step 0: 4813.8008
Training accuracy: 10.48%
Validation accuracy: 31.20%
Testing accuracy: 33.86%
loss at step 100: 49440.2969
Training accuracy: 74.53%
Validation accuracy: 74.73%
Testing accuracy: 81.57%
loss at step 200: 55139.9570
Training accuracy: 73.83%
Validation accuracy: 69.79%
Testing accuracy: 76.97%
loss at step 300: 40511.5234
Training accuracy: 77.83%
Validation accuracy: 74.00%
Testing accuracy: 81.92%
loss at step 400: 70940.6250
Training accuracy: 75.99%
Validation accuracy: 72.06%
Testing accuracy: 78.33%
loss at step 500: 39531.5781
Training accuracy: 78.71%
Validation accuracy: 73.46%
Testing accuracy: 81.31%
loss at step 600: 57561.5000
Training accuracy: 70.07%
Validation accuracy: 72.64%
Testing accuracy: 78.68%
loss at step 700: 94597.5391
Training accuracy: 72.80%
Validation accuracy: 70.91%
Testing accuracy: 78.17%
loss at step 800: 46973.8242
Training accuracy: 72.47%
Validation accuracy: 69.04%
Testing accuracy: 75.43%


In [19]:
#train_prediction = tf.nn.softmax(logits)
#valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases) 
#test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [20]:
batch_size = 100

graph_scd = tf.Graph()
with graph_scd.as_default():
    tf_X_train_scd = tf.placeholder(dtype=tf.float32, shape=[batch_size, X_train.shape[1]])
    tf_y_train_scd = tf.placeholder(dtype=tf.float32, shape =[batch_size, y_train.shape[1]])
    
    tf_weights_scd = tf.Variable(tf.truncated_normal(shape=[X_train.shape[1], y_train.shape[1]], dtype=tf.float32))
    tf_biases_scd = tf.Variable(tf.zeros(shape=y_train.shape[1]))
    
    logits = tf.matmul(a=tf_X_train_scd, b=tf_weights_scd) + tf_biases_scd
    
    loss_scd = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_y_train_scd))    
    tf_optimizer_scd = tf.train.GradientDescentOptimizer(learning_rate=0.05).minimize(loss_scd)
    
    tf_y_pred_scd = tf.nn.softmax(logits)
    
    

In [21]:

index = np.random.randint(low=0, high = 6706, size=batch_size)
batch_X_train = X_train[index,:]
batch_X_train.shape

(100, 784)

In [22]:
num_steps = 3001

with tf.Session(graph=graph_scd) as session:
    tf.initialize_all_variables().run()
    for i in range(num_steps):
        index = np.random.randint(low=0, high = 6706, size=batch_size)
        batch_X_train = X_train[index,:]
        batch_y_train = y_train[index,:]
        feed_dict = {tf_X_train_scd:batch_X_train, tf_y_train_scd:batch_y_train}
        _, l, predictions = session.run([tf_optimizer_scd, loss_scd, tf_y_pred_scd], feed_dict = feed_dict)
        if(i % 500 == 0):
            print("Loss at step %d: %f" % (i, l))
            print("Accuracy: %.1f%%" % accuracy(batch_y_train, predictions))
        

Loss at step 0: 7859.148926
Accuracy: 6.0%
Loss at step 500: 3373.341309
Accuracy: 83.0%
Loss at step 1000: 3761.447754
Accuracy: 80.0%
Loss at step 1500: 5316.018555
Accuracy: 74.0%
Loss at step 2000: 3386.474121
Accuracy: 84.0%
Loss at step 2500: 5053.129883
Accuracy: 81.0%
Loss at step 3000: 2525.650879
Accuracy: 85.0%


Now we will try to train Neural Network with Relu 


import tensorflow as tf
import numpy as np

# Data sets
IRIS_TRAINING = "iris_training.csv"
IRIS_TEST = "iris_test.csv"

# Load datasets.
training_set = X_y
test_set = X_y_test

# Specify that all features have real-value data
feature_columns = [tf.contrib.layers.real_valued_column("", dimension=4)]

# Build 3 layer DNN with 10, 20, 10 units respectively.
classifier = tf.contrib.learn.DNNClassifier(feature_columns=feature_columns,
                                            hidden_units=[10, 20, 10],
                                            n_classes=3,
                                            model_dir="/tmp/iris_model")

# Fit model.
classifier.fit(x=X_train,
               y=y_train,
               steps=2000)

# Evaluate accuracy.
accuracy_score = classifier.evaluate(x=X_test,
                                     y=y_test)["accuracy"]
print('Accuracy: {0:f}'.format(accuracy_score))

# Classify two new flower samples.
new_samples = np.array(
    [[6.4, 3.2, 4.5, 1.5], [5.8, 3.1, 5.0, 1.7]], dtype=float)
y = classifier.predict(new_samples)
print('Predictions: {}'.format(str(y)))

In [29]:
batch_size = 100

graph_scd = tf.Graph()
with graph_scd.as_default():
    tf_X_train_scd = tf.placeholder(dtype=tf.float32, shape=[batch_size, X_train.shape[1]])
    tf_y_train_scd = tf.placeholder(dtype=tf.float32, shape =[batch_size, y_train.shape[1]])
    
    tf_weights_scd = tf.Variable(tf.truncated_normal(stddev=0.1, shape=[X_train.shape[1], y_train.shape[1]], dtype=tf.float32))
    tf_biases_scd = tf.Variable(tf.ones(shape=y_train.shape[1]))
    
    logits = tf.matmul(a=tf_X_train_scd, b=tf_weights_scd) + tf_biases_scd
    print (logits)
    logits_relu = logits #tf.nn.relu(logits)
    print (logits_relu)
    loss_scd = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits_relu, tf_y_train_scd))    
    tf_optimizer_scd = tf.train.GradientDescentOptimizer(learning_rate=0.05).minimize(loss_scd)
    
    tf_y_pred_scd = tf.nn.softmax(logits_relu)

num_steps = 3001

with tf.Session(graph=graph_scd) as session:
    tf.initialize_all_variables().run()
    for i in range(num_steps):
        index = np.random.randint(low=0, high = 6706, size=batch_size)
        batch_X_train = X_train[index,:]
        batch_y_train = y_train[index,:]
        feed_dict = {tf_X_train_scd:batch_X_train, tf_y_train_scd:batch_y_train}
        _, l, predictions = session.run([tf_optimizer_scd, loss_scd, tf_y_pred_scd], feed_dict = feed_dict)
        if(i % 500 == 0):
            print("Loss at step %d: %f" % (i, l))
            print("Accuracy: %.1f%%" % accuracy(batch_y_train, predictions))
        

Tensor("add:0", shape=(100, 10), dtype=float32)
Tensor("add:0", shape=(100, 10), dtype=float32)
Loss at step 0: 509.861420
Accuracy: 15.0%
Loss at step 500: 4949.971680
Accuracy: 76.0%
Loss at step 1000: 16294.777344
Accuracy: 63.0%
Loss at step 1500: 6771.337402
Accuracy: 77.0%
Loss at step 2000: 5484.734863
Accuracy: 85.0%
Loss at step 2500: 2458.580566
Accuracy: 81.0%
Loss at step 3000: 3865.599121
Accuracy: 80.0%


In [28]:
batch_size = 50

graph_scd = tf.Graph()
with graph_scd.as_default():
    tf_X_train_scd = tf.placeholder(dtype=tf.float32, shape=[batch_size, X_train.shape[1]])
    tf_y_train_scd = tf.placeholder(dtype=tf.float32, shape =[batch_size, y_train.shape[1]])
    
    tf_weights_scd = tf.Variable(tf.truncated_normal(stddev=0.1, shape=[X_train.shape[1], 1024], dtype=tf.float32))
    tf_biases_scd = tf.Variable(tf.ones(shape=[1024]))
    print(tf_weights_scd)
    print(tf_biases_scd)
    logits = tf.matmul(a=tf_X_train_scd, b=tf_weights_scd) + tf_biases_scd
    print (logits)
    logits_relu = logits #tf.nn.relu(logits)
    print (logits_relu)
    
    # Let's add another layer for converting 1024 outputs to required number of classes i.e. y_train.shape[1]. This will be Readout layer which will use softmax function
    
    tf_weights_scd = tf.Variable(tf.truncated_normal(stddev=0.1, shape=[1024, y_train.shape[1]], dtype=tf.float32))
    tf_biases_scd = tf.Variable(tf.ones(shape=y_train.shape[1]))
    print(tf_weights_scd)
    print(tf_biases_scd)
    
    logits = tf.matmul(a=logits_relu, b=tf_weights_scd) + tf_biases_scd
    print (logits)
    
    loss_scd = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_y_train_scd))    
    tf_optimizer_scd = tf.train.GradientDescentOptimizer(learning_rate=0.05).minimize(loss_scd)
    
    tf_y_pred_scd = tf.nn.softmax(logits)

num_steps = 2000

with tf.Session(graph=graph_scd) as session:
    tf.initialize_all_variables().run()
    for i in range(num_steps):
        index = np.random.randint(low=0, high = X_train.shape[0], size=batch_size)
        batch_X_train = X_train[index,:]
        batch_y_train = y_train[index,:]
        feed_dict = {tf_X_train_scd:batch_X_train, tf_y_train_scd:batch_y_train}
        _, l, predictions = session.run([tf_optimizer_scd, loss_scd, tf_y_pred_scd], feed_dict = feed_dict)
        if(i % 500 == 0):
            print("Loss at step %d: %f" % (i, l))
            print("Accuracy: %.1f%%" % accuracy(batch_y_train, predictions))
        

Tensor("add:0", shape=(50, 1024), dtype=float32)
Tensor("add:0", shape=(50, 1024), dtype=float32)
Tensor("add_1:0", shape=(50, 10), dtype=float32)
Loss at step 0: 1234.371216
Accuracy: 12.0%
Loss at step 500: nan
Accuracy: 8.0%
Loss at step 1000: nan
Accuracy: 8.0%
Loss at step 1500: nan
Accuracy: 8.0%
